# README

#Setup

In [43]:
import numpy as np
import pandas as pd
import math
import itertools
import random
import torch
import os
import gzip
import json
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
from sentence_transformers import SentenceTransformer, util, losses, models
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForMaskedLM, DataCollatorForWholeWordMask, DataCollatorForLanguageModeling, pipeline
from transformers import AdamW, get_linear_schedule_with_warmup, TrainerCallback
from sklearn.model_selection import StratifiedKFold
import shutil
from datasets import load_metric
import gc
gc.enable()
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso, BayesianRidge, Perceptron, SGDRegressor

In [2]:
torch.cuda.is_available()


True

# Constants

In [3]:
BASE_PATH = '/home/masa1357/Dockerdata/gitfile/kaggle_clrp_1st_place_solution/'

In [4]:
# os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_num
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

In [5]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 28
seed_everything(seed=SEED)
MAX_LENGTH = 256

In [6]:
# fine-tuned model paths
# adjust path if you have saved the models in different directories
ALBERT_TRAINED_1 = os.path.join(BASE_PATH, 'models/albert-xxlarge-2-models')#OK
ALBERT_TRAINED_2 = os.path.join(BASE_PATH, 'models/albert-xxlarge-low-lr')#OK
ALBERT_TRAINED_3 = os.path.join(BASE_PATH, 'models/ALBERT_3/albert-xxlarge-all-data')#OK
DEBERTA_TRAINED_1 = os.path.join(BASE_PATH, 'models/deberta-large')#OK
DEBERTA_TRAINED_2 = os.path.join(BASE_PATH, 'models/deberta-large-low-lr')#OK
DEBERTA_TRAINED_3 = os.path.join(BASE_PATH, 'models/deberta-augmented-continued')#OK
ROBERTA_TRAINED_1 = os.path.join(BASE_PATH, 'models/roberta-large-two-models')#OK
ELECTRA_TRAINED_1 = os.path.join(BASE_PATH, 'models/electra-large')#OK

# Functions

In [7]:
def train_model(
    model_dir,
    out_dir,
    data,
    data_labels,
    test_data=None,
    test_labels=None,
    do_eval=False,
    do_epoch_eval=False,
    do_save_best=False,
    hyperparams={'bs': 16, 'lr': 1e-4, 'ep': 5, 'bias': False, 'init': None},
    cfg={'num_labels': 1, 'logging_steps': 500, 'is_multilabel': False, 'keep_layers': None}
    ):
  tokenizer = AutoTokenizer.from_pretrained(model_dir)
  
  train_encodings = tokenizer(data, truncation=True, padding=True, max_length=MAX_LENGTH)
  if test_data:
    test_encodings = tokenizer(test_data, truncation=True, padding=True, max_length=MAX_LENGTH)
  

  class LitDataset(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
          item['labels'] = torch.tensor(self.labels[idx])
          return item

      def __len__(self):
          return len(self.labels)

  train_dataset = LitDataset(train_encodings, data_labels)
  if test_data:
    test_dataset = LitDataset(test_encodings, test_labels)
  
  train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=hyperparams['bs'])
  training_steps = len(train_dataloader) * hyperparams['ep'] 
  warmup_steps = math.ceil(training_steps  * 0.06)

  training_args = TrainingArguments(
      output_dir=out_dir,          # output directory
      num_train_epochs=hyperparams['ep'],              # total number of training epochs
      per_device_train_batch_size=hyperparams['bs'],  # batch size per device during training
      per_device_eval_batch_size=1,   # batch size for evaluationing rate scheduler
      logging_dir='/tmp/logs',            # directory for storing logs
      logging_steps=cfg['logging_steps'],
      seed=SEED,
      weight_decay=hyperparams['weight_decay'],
      learning_rate=hyperparams['lr'],
      save_strategy='no'
  )
  config = AutoConfig.from_pretrained(
      model_dir,
      num_labels=cfg['num_labels'],
      hidden_dropout_prob=hyperparams['hidden_dropout'],
      attention_probs_dropout_prob=hyperparams['attention_probs_dropout'])
  model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=cfg['num_labels'])
  if hyperparams['init']:
    model = reinitialize_layers(model, hyperparams['init'])
  model.config = AutoConfig.from_pretrained(model_dir, num_labels=cfg['num_labels'])
  model.num_labels = cfg['num_labels']
  if cfg['keep_layers']:
    new_layers = torch.nn.ModuleList([layer_module for i, layer_module in enumerate(model.base_model.encoder.layer) if i in cfg['keep_layers']])
    model.base_model.encoder.layer = new_layers
    model.config.num_hidden_layers = len(cfg['keep_layers'])

  optimizer = AdamW(model.parameters(), correct_bias=hyperparams['bias'], lr=hyperparams['lr'])
  scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_training_steps=training_steps, num_warmup_steps=warmup_steps)
  device = "cuda:0"
  
  scores = []
  best_score = 1.0
  metric = load_metric("accuracy")

  class EvalCallback(TrainerCallback):
    def on_log(self, args, state, control, **kwargs):
      if do_save_best:
        model = kwargs['model']
        y_pred = predict_fast(init_model=model, tokenizer=tokenizer, data=test_data, num_labels=cfg['num_labels'], is_multilabel=cfg['is_multilabel'])
        model.train()
        curr_score = rms(test_labels, y_pred) if not cfg['is_multilabel'] else metric.compute(predictions=y_pred, references=test_labels)['accuracy']
        print('Score: ', curr_score)

        if len(scores) == 0 or min(scores) > curr_score:
          print(f'is min {curr_score} is smaller than {scores}')
          best_score = curr_score
          save_dir = os.path.join(out_dir, 'best')
          model.save_pretrained(save_dir)
          tokenizer.save_pretrained(save_dir)
          with open(os.path.join(save_dir, 'hyperparams.txt'), 'w') as f:
            hyperparams['score'] = curr_score
            hyperparams['step'] = state.global_step
            hyperparams['trainset_size'] = len(data_labels)
            f.write(json.dumps(hyperparams))
        scores.append(curr_score)

  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      optimizers=(optimizer, scheduler),
      callbacks=[EvalCallback]             # evaluation dataset
  )

  trainer.train()

  if not do_save_best:
    model.save_pretrained(out_dir)
    tokenizer.save_pretrained(out_dir)
  print('Training done')

  if do_save_best:
    del model
    gc.collect()
    return min(scores)

In [8]:
def train_cv_v2(model_dir, out_dir, fold_dir, hyperparams, cfg, kfolds=[0, 1, 2, 3, 4, 5], continue_training=False, deduplicate=False, soft_label_model=None):
  scores = []
  for fold in kfolds:
    train_df = pd.read_csv(fold_dir + '/train_fold_' + str(fold) + '.csv')
    val_df = pd.read_csv(fold_dir + '/val_fold_' + str(fold) + '.csv')
    if deduplicate:
      train_df = train_df.drop_duplicates(subset=['excerpt'])
    train_tx = [str(t) for t in train_df.excerpt.values]
    train_sc = [float(t) for t in train_df.target.values]
    val_tx = [str(t) for t in val_df.excerpt.values]
    val_sc = [float(t) for t in val_df.target.values]

    model_out_dir = out_dir + '/model_fold_' + str(fold)
    if continue_training:
      final_model_dir = model_dir + '/model_fold_' + str(fold) + '/best'
    else:
      final_model_dir = model_dir
    
    if cfg['soft_labels'] == 'add':
      preds = predict_fast(final_model_dir, train_tx)
      train_tx = train_tx + train_tx
      train_sc = train_sc + preds
    if cfg['soft_labels'] == 'only':
      preds = predict_fast(final_model_dir, train_tx)
      train_tx = train_tx
      train_sc = preds
    if soft_label_model and cfg['soft_labels'] == 'add':
      preds = predict_fast(soft_label_model + '/model_fold_' + str(fold) + '/best', train_tx)
      train_sc = train_sc + preds
      train_tx = train_tx + train_tx
    if soft_label_model and cfg['soft_labels'] == 'only':
      preds = predict_fast(soft_label_model + '/model_fold_' + str(fold) + '/best', train_tx)
      train_sc = preds
      train_tx = train_tx
      
    best_score = train_model(
        model_dir=final_model_dir,
        out_dir=model_out_dir,
        data=train_tx,
        data_labels=train_sc,
        test_data=val_tx,
        test_labels=val_sc,
        do_save_best=True,
        hyperparams=hyperparams,
        cfg=cfg
      )
    scores.append(best_score)
  cv_score = np.mean(scores)
  with open(out_dir + '/eval.txt', 'w') as f:
    f.write('CV score is ' + str(cv_score))

In [9]:
def predict_fast(model_name=None, data=None, init_model=None, tokenizer=None, num_labels=1, is_multilabel=False, output_logits=False, use_softmax=False):
  device = "cuda:0"
  tokenizer = AutoTokenizer.from_pretrained(model_name) if model_name else tokenizer
  config = AutoConfig.from_pretrained(model_name, num_labels=num_labels) if model_name else None
  model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config) if model_name else init_model
  model.to(device)
  model.eval()
  y_pred = []
  batches = chunks(data, 32)
  for batch in tqdm(batches):
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LENGTH)
    input_ids = inputs['input_ids'].to(device)
    attention = inputs['attention_mask'].to(device)
    inputs = {
        'input_ids': input_ids,
        'attention_mask': attention
    }
    with torch.no_grad():        
          outputs = model(**inputs)
    if not use_softmax:
      logits = outputs[0].detach().cpu().numpy().squeeze().tolist()
    else:
      logits = nn.functional.softmax(outputs.logits, dim=-1).detach().cpu().numpy().squeeze().tolist()
    if is_multilabel and not output_logits:
      logits = np.argmax(logits, axis=-1)
    y_pred.extend(logits)
  del model
  gc.collect()
  return y_pred

In [10]:
def get_oof_predictions(model_dirs, fold_dir, out_dir, kfolds=[0,1,2,3,4,5]):
  df = pd.DataFrame()
  
  for fold in kfolds:
    val_df = pd.read_csv(fold_dir + '/val_fold_' + str(fold) + '.csv')
    val_tx = [str(t) for t in val_df.excerpt.values]
    val_sc = [float(t) for t in val_df.target.values]
    fold_df = pd.DataFrame()
    fold_df['fold'] = [fold for v in val_sc]
    fold_df['excerpt'] = val_tx
    fold_df['target'] = val_sc
    fold_df['id'] = val_df['id']

    for model in model_dirs:
      final_model_dir = model + '/model_fold_' + str(fold) + '/best'
      model_name = model.split('/')[-1]
      preds = predict_fast(final_model_dir, val_tx)
      fold_df[model_name] = preds
    df = df.append(fold_df, ignore_index=True)
  
  df.to_csv(out_dir)  

In [ ]:
def get_bin_stratified(df, n_bins=20, n_splits=5):
    df['bin'] = pd.cut(df.target, n_bins, labels=[i for i in range(n_bins)])
    
    df['fold'] = np.nan

    skf = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)
    gen_skf = skf.split(df.id, y=df.bin)

    for fold, (idx_train, idx_val) in enumerate(gen_skf):
        df.loc[idx_val, 'fold'] = fold
    
    df['fold'] = df['fold'].astype('int8')

In [13]:
def rms(y_actual, y_predicted):
  return mean_squared_error(y_actual, y_predicted, squared=False)

In [52]:
# FIX errno
import errno
# FIX dump
from joblib import dump


In [53]:
def postprocess_predictions(predictions, bin_predictions, bin_averages, threshold=0.58):
  new_predictions = []
  for idx, p in enumerate(predictions):
    if abs(p - bin_averages[bin_predictions[idx][0]]) > 0.5 and np.argmax(bin_predictions[idx][1]) > threshold:
      new_predictions.append(np.mean([p, bin_averages[bin_predictions[idx][0]]]))
    else:
      new_predictions.append(p)
  return new_predictions

In [54]:
from itertools import combinations
from sklearn.metrics import mean_squared_error

def find_best_stack(avg_df, ridge_names, drop_models=False):
    best_rmse = float('inf')
    best_combination = None

    # Iterate over all possible combinations of models
    for r in range(1, len(ridge_names)+1):
        for subset in combinations(ridge_names, r):
            # Calculate the mean prediction
            mean_pred = avg_df[list(subset)].mean(axis=1)
            
            # Calculate the RMSE of the mean prediction
            rmse = mean_squared_error(avg_df['target'], mean_pred, squared=False)
            
            # If this RMSE is better than the current best, update the best RMSE and combination
            if rmse < best_rmse:
                best_rmse = rmse
                best_combination = subset

    return best_combination


In [55]:
# BUG don't defined
# - [x] 1.errno
# - [x] 2.dump
# - [x] 3.find_best_stack
# - [x] 4.postprocess_predictions
# - [x] 5.rms
def train_leaky_ensembler(oof_dir, model_names, out_dir, kfolds=[0,1,2,3,4,5], model_bins=[], clf='ridge', find_opt_avg=False, bin_avg_dir=None, use_postprocessing=False):
  df = pd.read_csv(oof_dir)

  if find_opt_avg:
    msk = np.random.rand(len(df)) < 0.2
    df_test = df[msk].reset_index(drop=True)
    df = df[~msk].reset_index(drop=True)
    
  get_bin_stratified(df, n_splits=6)

  results = []
  if find_opt_avg:
    avg_df = pd.DataFrame()
    avg_df['target'] = [float(f) for f in df_test['target']]
  for fold in kfolds:
    train_df = df.loc[df.fold!=fold].reset_index(drop=True)
    val_df = df.loc[df.fold==fold].reset_index(drop=True)
    
    train_tx = [str(t) for t in train_df.excerpt.values]
    val_tx = [str(t) for t in val_df.excerpt.values]
    val_sc = [float(f) for f in val_df.target.values]
    train_sc = [float(f) for f in train_df.target.values]

    train_predictions = []
    val_predictions = []
    avg_predictions = []

    if len(model_bins) > 0 and not use_postprocessing:
      for model_name in model_bins:
        preds = [json.loads(p) for p in train_df[model_name].values]
        preds_val = [json.loads(p) for p in val_df[model_name].values]
        if bin_avg_dir:
          with open(bin_avg_dir, 'r') as f:
            averages = json.loads(f.read())
          preds = [averages[np.argmax(p)] for p in preds]
          preds_val = [averages[np.argmax(p)] for p in preds_val]

        train_predictions.append(preds)
        val_predictions.append(preds_val)
    
    for model_name in model_names:
      preds = [float(f) for f in train_df[model_name].values]
      train_predictions.append(np.array(preds))
      preds_val = [float(f) for f in val_df[model_name].values]
      val_predictions.append(np.array(preds_val))
      if find_opt_avg:
        preds_avg = [float(f) for f in df_test[model_name].values]
        avg_predictions.append(np.array(preds_avg))
    
    X = np.column_stack(train_predictions)
    
    if clf == 'ridge':
      clf = Ridge(alpha=1.0)
    elif clf == 'linearsvr':
      clf = LinearSVR(max_iter=1000000)
    elif clf == 'svr':
      clf = SVR()
    elif clf == 'kernel':
      clf = KernelRidge()
    elif clf == 'gbr':
      clf = GradientBoostingRegressor()
    elif clf == 'linear':
      clf = LinearRegression()
    elif clf == 'lasso':
      clf = Lasso()
    elif clf == 'bayes':
      clf = BayesianRidge()
    elif clf == 'perceptron':
      clf = SGDRegressor()
    
    clf.fit(X, train_sc)

    final_out = out_dir + '/model_fold_' + str(fold) + '/'
    if not os.path.exists(os.path.dirname(final_out)):
      try:
          os.makedirs(os.path.dirname(final_out))
      except OSError as exc: # Guard against race condition
          if exc.errno != errno.EEXIST:
              raise
    dump(clf, final_out + 'ridge_model.joblib')

    Y = np.column_stack(val_predictions)

    y_preds = clf.predict(Y)
    if use_postprocessing:
      preds_val = [json.loads(p) for p in val_df[model_bins[0]].values]
      with open(bin_avg_dir, 'r') as f:
            averages = json.loads(f.read())
      preds_val_bins = [np.argmax(p) for p in preds_val]
      zipped = list(zip(preds_val_bins, preds_val))
      y_preds = postprocess_predictions(y_preds, zipped, averages)

    score = rms(val_sc, y_preds)
    print('Score is: ', score)
    results.append(score)

    if find_opt_avg:
      Y_test = np.column_stack(avg_predictions)
      y_preds_test = clf.predict(Y_test)
      avg_df['fold_' + str(fold)] = y_preds_test
  
  if find_opt_avg:
    ridge_names = ['fold_' + str(fold) for fold in range(kfolds)]
    print(find_best_stack(avg_df, ridge_names, drop_models=False))

  with open(out_dir + '/eval.txt', 'w') as f:
    mean = np.mean(results)
    print('CV ist: ', mean)
    f.write('CV is: ' + str(mean))

In [12]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Pretraining models

In [14]:
# Load the pseudo-labeled training data for pretraining models
train_df = pd.read_csv(os.path.join(BASE_PATH, 'data/training/predicted/predicted.csv'))
train_tx = [str(t) for t in train_df.excerpt.values]
train_sc = [float(t) for t in train_df.target.values]

In [15]:
# Load the entire training set from the original competition for validation during pretraining
val_df = pd.read_csv(os.path.join(BASE_PATH, 'data/training/original/train.csv'))
val_tx = [str(t) for t in train_df.excerpt.values]
val_sc = [float(t) for t in train_df.target.values]

In [16]:
# Train an ALBERT model

model_name = 'albert-xxlarge-v2'
hyperparams = {
  'bs': 3,
  'lr': 9e-6,
  'weight_decay': 0.01,
  'ep': 5,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.07,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 60,
  'keep_layers': None,
  'soft_labels': None
}

ALBERT_PRETRAINED = os.path.join(BASE_PATH, 'models/albert-xxlarge-no-cv')

In [ ]:
train_model(
    model_dir=model_name,
    out_dir=ALBERT_PRETRAINED,
    data=train_tx,
    data_labels=train_sc,
    test_data=val_tx,
    test_labels=val_sc,
    do_save_best=True,
    hyperparams=hyperparams,
    cfg=cfg
)

Some weights of the model checkpoint at albert-xxlarge-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-xxlarge-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

Step,Training Loss
60,8.637400
120,3.487500
180,1.895200
240,1.094500
300,0.712400
360,0.465100
420,0.373500
480,0.322300
540,0.274300
600,0.296700


143it [09:34,  4.01s/it]


Score:  2.4724326828566308
is min 2.4724326828566308 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
143it [09:33,  4.01s/it]


...

Score:  0.0994916672241441
is min 0.0994916672241441 is smaller than [2.4724326828566308, 1.6261113322019147, 1.2382114282735124, 0.8307748337080949, 0.6699659748066764, 0.6095859441192493, 0.49570119262401324, 0.5951307450469295, 0.38556877974187387, 0.46415313261570956, 0.30167225140091514, 0.8091544663570209, 0.35171971905965443, 0.34414852981525146, 0.37126143932883837, 0.2598407135895769, 0.2228464173286103, 0.2510136222937503, 0.24481901878477488, 0.23501769343129786, 0.2084207408779208, 0.27096740083214366, 0.22529955129597068, 0.2030326020444604, 0.1855980143156472, 0.2191443796511697, 0.16730965007322166, 0.18291989960003677, 0.2019890943621976, 0.17601716703277046, 0.1709428050130544, 0.16370083961862703, 0.14341060540823583, 0.20681427353621434, 0.19878128240589682, 0.13941787429397656, 0.24509122909784195, 0.14509823556475573, 0.14945943201785525, 0.1490927537202429, 0.13095789812624467, 0.13947510328993964, 0.17832777033590355, 0.1339956069155411, 0.18521502810153598, 0.13

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
143it [09:29,  3.98s/it]

Score:  0.10300967809098473


Training done


0.0994916672241441

In [16]:
# Train a DEBERTA model
model_name = 'microsoft/deberta-large'
hyperparams = {
  'bs': 3,
  'lr': 9e-6,
  'weight_decay': 0.1,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 20,
  'keep_layers': None,
  'soft_labels': None
}

DEBERTA_PRETRAINED = os.path.join(BASE_PATH, 'models/deberta-large-augmented')

train_model(
    model_dir=model_name,
    out_dir=DEBERTA_PRETRAINED,
    data=train_tx,
    data_labels=train_sc,
    test_data=val_tx,
    test_labels=val_sc,
    do_save_best=True,
    hyperparams=hyperparams,
    cfg=cfg
)

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['classifier.bias', 'pooler.d

Step,Training Loss
20,1.760600
40,1.736800
60,1.241500
80,0.623100
100,0.517300
120,0.449100
140,0.382400
160,0.399000
180,0.337800
200,0.278900


143it [01:33,  1.54it/s]


Score:  1.3350963896607595
is min 1.3350963896607595 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
143it [01:35,  1.50it/s]


...

Score:  0.23005158225276462


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
143it [01:41,  1.40it/s]

Score:  0.22368336831050722


Training done


0.12351018220386077

In [17]:
# Train a RoBERTa model
model_name = 'roberta-large'
hyperparams = {
  'bs': 8,
  'lr': 1e-5,
  'weight_decay': 0.01,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

ROBERTA_PRETRAINED = os.path.join(BASE_PATH, 'models/roberta-large-augmented')

train_model(
    model_dir=model_name,
    out_dir=ROBERTA_PRETRAINED,
    data=train_tx,
    data_labels=train_sc,
    test_data=val_tx,
    test_labels=val_sc,
    do_save_best=True,
    hyperparams=hyperparams,
    cfg=cfg
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should 

Step,Training Loss
10,1.187000
20,1.137200
30,0.932700
40,0.808300
50,0.688600
60,0.671500
70,0.471300
80,0.447200
90,0.411200
100,0.366700


143it [01:10,  2.02it/s]


Score:  1.0748452290566466
is min 1.0748452290566466 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
143it [01:17,  1.86it/s]


...

Score:  0.20921944937230724


143it [01:15,  1.90it/s]

Score:  0.3673777991611483


Training done


0.12704855938069418

In [18]:
# Train an ELECTRA model
model_name = 'google/electra-large-discriminator'
hyperparams = {
  'bs': 4,
  'lr': 8e-6,
  'weight_decay': 0.1,
  'ep': 7,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

ELECTRA_PRETRAINED = os.path.join(BASE_PATH, 'models/electra-large-augmented')

train_model(
    model_dir=model_name,
    out_dir=ELECTRA_PRETRAINED,
    data=train_tx,
    data_labels=train_sc,
    test_data=val_tx,
    test_labels=val_sc,
    do_save_best=True,
    hyperparams=hyperparams,
    cfg=cfg
)

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

Step,Training Loss
10,1.987400
20,1.363900
30,2.017500
40,1.608900
50,1.570000
60,1.271300
70,1.195500
80,1.213800
90,0.811700
100,0.916400


143it [01:08,  2.10it/s]


Score:  1.3423287768124137
is min 1.3423287768124137 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
143it [01:10,  2.03it/s]


...

Score:  0.24099914287557161


143it [01:15,  1.89it/s]

Score:  0.16334453935244994


Training done


0.1256021698283635

# Training models

In total, I trained 3 deberta-large, 1 roberta-large, 3 albert-xxlarge and 1 electra-large model for my winning submission.



In [ ]:
# Training the ALBERT models

In [17]:
# albert 1
model_name = os.path.join(ALBERT_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 9e-6,
  'weight_decay': 0.01,
  'ep': 5,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.07,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = ALBERT_TRAINED_1

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_16155/202785219.py:75: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.271600
20,0.262700
30,0.200800
40,0.241800
50,0.304900
60,0.312600
70,0.207700
80,0.223600
90,0.185500
100,0.229600


15it [00:56,  3.79s/it]


Score:  0.45585228717880494
is min 0.45585228717880494 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:58,  3.93s/it]
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

Score:  0.4501226816267056


15it [00:58,  3.90s/it]


Score:  0.45470366972907184


15it [00:58,  3.92s/it]


Score:  0.45470366972907184
Training done


In [18]:
# albert 2
model_name = os.path.join(ALBERT_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 7e-6,
  'weight_decay': 0.07,
  'ep': 5,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = ALBERT_TRAINED_2

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.271500
20,0.264000
30,0.197000
40,0.243000
50,0.304600
60,0.315500
70,0.208100
80,0.224900
90,0.186900
100,0.226100


15it [00:58,  3.91s/it]


Score:  0.4557647680114362
is min 0.4557647680114362 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:59,  3.96s/it]
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

Score:  0.46044736911062817


15it [00:58,  3.88s/it]


Score:  0.46339441532776


15it [00:58,  3.88s/it]


Score:  0.46339441532776
Training done


In [19]:
# albert 3
# albert 3 is special it is trained on all training data without evaluation.
model_name = os.path.join(ALBERT_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 9e-6,
  'weight_decay': 0.1,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 600,
  'keep_layers': None,
  'soft_labels': None
}

train_df = pd.read_csv(os.path.join(BASE_PATH, 'data/training/original/train.csv'))
train_tx = [str(t) for t in train_df.excerpt.values]
train_sc = [float(t) for t in train_df.target.values]

out_dir = ALBERT_TRAINED_3


train_model(
   model_dir=model_name,
   out_dir=out_dir,
   data=train_tx,
   data_labels=train_sc,
   hyperparams=hyperparams,
   cfg=cfg
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
600,0.230100
1200,0.138200
1800,0.070500


Training done


In [21]:
# Training the deberta models
DEBERTA_PRETRAINED = os.path.join(BASE_PATH, 'models/deberta-large-augmented')

In [22]:
# deberta 1
model_name = os.path.join(DEBERTA_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 9e-6,
  'weight_decay': 0.1,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = DEBERTA_TRAINED_1

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.309600
20,0.283600
30,0.191800
40,0.237300
50,0.294900
60,0.313900
70,0.219700
80,0.228100
90,0.190800
100,0.232100


15it [00:09,  1.58it/s]


Score:  0.47703367801001606
is min 0.47703367801001606 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:09,  1.57it/s]


...

Score:  0.6054470414919079


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.46it/s]


Score:  0.4653180803615165


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.45it/s]


Score:  0.5575311774956899
Training done


In [23]:
# deberta 2
model_name = os.path.join(DEBERTA_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 7e-6,
  'weight_decay': 0.1,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = DEBERTA_TRAINED_2

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.310600
20,0.290100
30,0.193800
40,0.239400
50,0.299200
60,0.312000
70,0.218000
80,0.231000
90,0.195000
100,0.228300


15it [00:10,  1.49it/s]


Score:  0.47585135438270953
is min 0.47585135438270953 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.47it/s]


...

Score:  0.5831849226135195


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.46it/s]


Score:  0.49148617212427675


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.45it/s]


Score:  0.5183273958241672
Training done


In [24]:
# deberta 3
# This deberta model was trained on data sampled using bootstrapping instead of cross validation
# Only models trained on 2 folds/bags were used in the final submission
model_name = os.path.join(DEBERTA_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 9e-6,
  'weight_decay': 0.08,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = DEBERTA_TRAINED_3

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg,
    kfolds=[0,1]
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.309600
20,0.283600
30,0.191800
40,0.237300
50,0.294900
60,0.313900
70,0.219700
80,0.228100
90,0.190800
100,0.232100


15it [00:10,  1.48it/s]


Score:  0.47703367801001606
is min 0.47703367801001606 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.46it/s]


...

Score:  0.5462523296635715


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.45it/s]


Score:  0.5076969836219132


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:10,  1.44it/s]


Score:  0.5124533041809607
Training done


In [26]:
# Training the ELECTRA model
ELECTRA_PRETRAINED = os.path.join(BASE_PATH, 'models/electra-large-augmented')

In [27]:
# electra 1
model_name = os.path.join(ELECTRA_PRETRAINED, 'best')
hyperparams = {
  'bs': 3,
  'lr': 8e-6,
  'weight_decay': 0.1,
  'ep': 5,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = ELECTRA_TRAINED_1

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.511600
20,0.435300
30,0.253000
40,0.225700
50,0.304800
60,0.321100
70,0.205600
80,0.224100
90,0.181800
100,0.220100


15it [00:06,  2.15it/s]


Score:  0.5009080389472447
is min 0.5009080389472447 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:07,  2.13it/s]


...

Score:  0.6233701182757613


15it [00:07,  2.00it/s]


Score:  0.5335168803972156


15it [00:07,  1.96it/s]


Score:  0.5335168803972156
Training done


In [30]:
# Training the RoBERTa model

ROBERTA_PRETRAINED = os.path.join(BASE_PATH, 'models/roberta-large-augmented')

In [31]:
# roberta 1
model_name = os.path.join(ROBERTA_PRETRAINED, 'best')
hyperparams = {
  'bs': 8,
  'lr': 1e-5,
  'weight_decay': 0.1,
  'ep': 4,
  'bias': True,
  'init': None,
  'hidden_dropout': 0.1,
  'attention_probs_dropout': 0.1
}
cfg = {
  'num_labels': 1,
  'is_multilabel': False,
  'logging_steps': 10,
  'keep_layers': None,
  'soft_labels': None
}

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = ROBERTA_TRAINED_1

train_cv_v2(
    model_dir=model_name,
    out_dir=out_dir,
    fold_dir=fold_dir,
    hyperparams=hyperparams,
    cfg=cfg
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.376200
20,0.286500
30,0.316100
40,0.208400
50,0.233800
60,0.264200
70,0.224200
80,0.243700
90,0.249700
100,0.341300


15it [00:06,  2.16it/s]


Score:  0.5444286720350329
is min 0.5444286720350329 is smaller than []


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
15it [00:06,  2.16it/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

Score:  0.7681509789406549


15it [00:07,  1.97it/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Score:  0.7815393209021655


15it [00:07,  1.95it/s]


Score:  0.7868377045104237
Training done


# Stacking

In [38]:
model_dirs = [
    ALBERT_TRAINED_1,
    DEBERTA_TRAINED_1,
    ALBERT_TRAINED_2,
    DEBERTA_TRAINED_1,
    ROBERTA_TRAINED_1,
    ELECTRA_TRAINED_1
]

fold_dir = os.path.join(BASE_PATH, 'data/training/cv')
out_dir = os.path.join(BASE_PATH,  'data/training/oof')

In [40]:
get_oof_predictions(model_dirs=model_dirs, fold_dir=fold_dir, out_dir=out_dir)

15it [00:57,  3.86s/it]
15it [00:10,  1.40it/s]
15it [01:00,  4.02s/it]
15it [00:10,  1.41it/s]
15it [00:07,  1.95it/s]
15it [00:07,  1.96it/s]
/tmp/ipykernel_16155/3398867116.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fold_df, ignore_index=True)
15it [00:59,  3.94s/it]
15it [00:10,  1.42it/s]
15it [00:58,  3.93s/it]
15it [00:10,  1.44it/s]
15it [00:07,  1.98it/s]
15it [00:07,  1.98it/s]
/tmp/ipykernel_16155/3398867116.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fold_df, ignore_index=True)
15it [00:58,  3.90s/it]
15it [00:10,  1.44it/s]
15it [00:58,  3.90s/it]
15it [00:10,  1.45it/s]
15it [00:07,  2.00it/s]
15it [00:07,  2.01it/s]
/tmp/ipykernel_16155/3398867116.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [60]:
model_names_ensemble_1 = [
    ALBERT_TRAINED_1.split('/')[-1],
    DEBERTA_TRAINED_1.split('/')[-1],
    ALBERT_TRAINED_2.split('/')[-1],
    DEBERTA_TRAINED_1.split('/')[-1],
    ROBERTA_TRAINED_1.split('/')[-1],
    ELECTRA_TRAINED_1.split('/')[-1],      
]

#model_names_ensemble_2 = model_names_ensemble_1[:-1]
model_names_ensemble_2 = model_names_ensemble_1

oof_dir = os.path.join(BASE_PATH, 'data/training/oof')

out_dir_ensemble_1 = os.path.join(BASE_PATH, 'models/electra-larger-ensemble')
out_dir_ensemble_2 = os.path.join(BASE_PATH, 'models/huge-ensemble')

In [61]:
# train ensemble 1
# [ ] DO NOW!!
train_leaky_ensembler(oof_dir=oof_dir, model_names=model_names_ensemble_1, out_dir=out_dir_ensemble_1)

Score is:  0.45536545111612503
Score is:  0.44102645950683217
Score is:  0.44455115946987456
Score is:  0.43465356645860115
Score is:  0.40216299968558455
Score is:  0.4159162886413556
CV ist:  0.43227932081306214


In [62]:
# train ensemble 2
train_leaky_ensembler(oof_dir=oof_dir, model_names=model_names_ensemble_2, out_dir=out_dir_ensemble_2)

Score is:  0.45536545111612503
Score is:  0.44102645950683217
Score is:  0.44455115946987456
Score is:  0.43465356645860115
Score is:  0.40216299968558455
Score is:  0.4159162886413556
CV ist:  0.43227932081306214


You have finished training the models.